In [6]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from chatbot import ChatBot, load_api_key
from OktSentiment import Sentiment
from tarot import TarotBot

app = Flask(__name__)
CORS(app)

# ChatBot 인스턴스 생성
api_key = load_api_key()
if not api_key:
    raise ValueError("API key not found or invalid. Please check your api_key.txt file.")
chatbot = ChatBot(api_key, 'c:/Users/SMHRD/원인-대처종합.csv')  # CSV 파일 경로를 적절히 수정하세요
#C:\Users\SMHRD\Documents\카카오톡 받은 파일\원인-대처종합.csv //c:/Users/SMHRD/원인-대처종합.csv
# Sentiment 인스턴스 생성
sentiment_analyzer = Sentiment()

@app.route("/chatbot", methods=["POST"])
def chat():
    data = request.json
    print("Received data:", data)
    user_nick = data.get("user_nick")
    user_input = data.get("input")
    
    if user_input:
        try:
            response = chatbot.process_input(user_nick, user_input)
            if response.get("isProblem"):
                # 형태소 분석 및 감정 분석 수행
                sentiment_results = sentiment_analyzer.parse([user_input])
                # 결과가 딕셔너리 형태로 바꾸고 소수점 소수점 둘쨰자리까지
                if 'polarity' in sentiment_results and len(sentiment_results['polarity']) > 0:
                    sentiment_results = sentiment_results['polarity'][0]
                    sentiment_results = {key: round(value * 100, 2) for key, value in sentiment_results.items()}

                response.update({
                    "sentiment": sentiment_results
                })

            return jsonify(response)
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            return jsonify({"error": f"An error occurred while processing your request: {str(e)}"}), 500
    else:
        return jsonify({"error": "No input provided"}), 400

# 타로카드 라우트
@app.route("/api/interpret",methods=["POST"])

def tarot_card():
    data = request.json
    cards = data.get("cards")
    user_input = data.get("user_input")
    user_select = data.get("user_select")
    print("recived data: ",data)
    if not cards or len(cards) != 3:
        return jsonify({"error": "카드를 다시 선택해 주세요."}),400
    
    key_path = "c:\\Users\\SMHRD\\api.txt"
    try:
        tarot = TarotBot(api_key_path=key_path)
        if user_input:
            print(user_input)
            response = tarot.interpret_cards(user_input,cards)
        else:
            response = tarot.general_reading(user_select,cards)
        print("response: ",response.get_json())
        return response

    except Exception as e:
        print("Error",str(e))
        return jsonify({"error":str(e)}),500

if __name__ == "__main__":
    app.run(port=7000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:7000
Press CTRL+C to quit
127.0.0.1 - - [04/Sep/2024 16:33:23] "OPTIONS /chatbot HTTP/1.1" 200 -


Received data: {'user_nick': '12', 'input': '취업준비가 너무 힘들어요'}
Processing input: user_nick=12, user_input=취업준비가 너무 힘들어요
Is problem flag: True
고민임


127.0.0.1 - - [04/Sep/2024 16:33:25] "POST /chatbot HTTP/1.1" 200 -


Extracted reason: '취업/진로'
Problem reason: 취업/진로
정렬된 형태소 분석 결과: ['취업/NNG', '준비/NNG', '가/JKS', '너무/MAG', '힘들어요/VA']


...

Extracted reason: '취업/진로'
Problem reason: 취업/진로
정렬된 형태소 분석 결과: ['취업/NNG', '준비/NNG', '가/JKS', '너무/MAG', '힘들어요/VA']


127.0.0.1 - - [04/Sep/2024 16:34:19] "OPTIONS /api/interpret HTTP/1.1" 200 -


recived data:  {'user_select': '오늘의 운세', 'cards': ['Temperance', 'The Emperor', 'Four of Swords'], 'user_input': '취업준비가 너무 힘들어요'}
취업준비가 너무 힘들어요


127.0.0.1 - - [04/Sep/2024 16:34:23] "POST /api/interpret HTTP/1.1" 200 -


response:  {'answer': '카드의 의미:\n- Temperance: 조화와 균형\n- The Emperor: 권위와 구조\n- Four of Swords: 휴식과 재충전\n\n종합 해석:\n취업준비가 너무 힘들다고 느끼는 당신에게 타로카드는 균형, 구조, 그리고 휴식을 강조하고 있습니다. Temperance 카드는 조화와 균형을 찾으라고 조언합니다. The Emperor 카드는 체계적인 계획과 규율을 통해 안정감을 찾으라는 메시지를 전달합니다. 마지막으로, Four of Swords는 잠시 휴식을 취하고 재충전을 하라는 신호입니다. 과도한 스트레스는 오히려 역효과를 낼 수 있으니, 잠시 멈추고 자신을 돌보며 에너지를 회복하는 것이 중요합니다. 균형 잡힌 접근과 체계적인 계획을 통해 더 나은 결과를 얻을 수 있을 것입니다.'}
